In [ ]:
!pip install langchain google-cloud-aiplatform google-auth > /dev/null

In [ ]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/generativeaitrial-trialLC.json'

In [ ]:
from langchain.llms import VertexAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatVertexAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage,
    SystemMessage
)

In [ ]:
llm = VertexAI(temperature=0.7)

In [ ]:
chatLLM = ChatVertexAI(temperature=0.7)

In [ ]:
def generate_response(system_prompt, 
                      user_prompt,model=chatLLM):
    messages = []
    messages.append(SystemMessage(content=system_prompt))
    messages.append(HumanMessage(content=user_prompt))

    # Send the API request
    response = model(messages)

    # Get the reply from the API response
    return response

In [ ]:
def generate_file(filename, 
                  model=chatLLM, 
                  filepaths_string=None, 
                  shared_dependencies=None, 
                  prompt=None):
    
    filecode = generate_response(model, 
        f"""You are an AI developer who is trying to write a program that will generate code for the user based on their intent.
        
    the app is: {prompt}

    the files we have decided to generate are: {filepaths_string}

    the shared dependencies (like filenames and variable names) we have decided on are: {shared_dependencies}
    
    only write valid code for the given filepath and file type, and return only the code.
    do not add any other explanation, only return valid code for that file type.
    """,
        f"""
    We have broken up the program into per-file generation. 
    Now your job is to generate only the code for the file {filename}. 
    Make sure to have consistent filenames if you reference other files we are also generating.
    
    Remember that you must obey 3 things: 
       - you are generating code for the file {filename}
       - do not stray from the names of the files and the shared dependencies we have decided on
       - MOST IMPORTANT OF ALL - the purpose of our app is {prompt} - every line of code you generate must be valid code. Do not include code fences in your response, for example
    
    Bad response:
    ```python 
    print("hello world")
    ```
    
    Good response:
    print("hello world")
    
    Begin generating the code now.

    """,
    )

    return filename, filecode

In [ ]:
prompt ="""A Python script that creates GUI which has text boxes that accepts the 
name of the item purchased, and the quantity. 
- When submit button is then it calculates the 
total cost based on the price of the item. Locate the submit button at the bottom of the gui. 
- Write the code to connect with the price database to get the price of the item that is purchased.
- GUI also has button to create invoice copy based on the item, quantity purchased and price. 
- Then there is a close button, which ask where invoice copy is required by creating a popup. 
After getting the user input, either closes or prints the invoice and then closes.
Important Details:
- Code has to run in a gui environment
- Write the test code for checking if the cost calculation and the 
invoice is generated correctly.
"""

In [ ]:
def write_file(filename, filecode, directory):
    # Output the filename in blue color
    print("\033[94m" + filename + "\033[0m")
    print(filecode)
    
    file_path = os.path.join(directory, filename)
    dir = os.path.dirname(file_path)

    # Check if the filename is actually a directory
    if os.path.isdir(file_path):
        print(f"Error: {filename} is a directory, not a file.")
        return

    os.makedirs(dir, exist_ok=True)

    # Open the file in write mode
    with open(file_path, "w") as file:
        # Write content to the file
        file.write(filecode)

In [ ]:
def clean_dir(directory):
    # Check if the directory exists
    if os.path.exists(directory):
        # If it does, iterate over all files and directories
        for dirpath, _, filenames in os.walk(directory):
            for filename in filenames:
              os.remove(os.path.join(dirpath, filename))
    else:
        os.makedirs(directory, exist_ok=True)

In [ ]:
filepaths_string = generate_response(system_prompt="""You are an AI developer who is trying to write a program that will generate code for the user based on their intent.
        
    When given their intent, create a complete, exhaustive list of filepaths that the user would write to make the program.
    
    only list the filepaths you would write, and return them as a python list of strings. 
    do not add any other explanation, only return a python list of strings.
    """,user_prompt = prompt,model=chatLLM
    )

In [ ]:
filepaths_string

AIMessage(content='Here is a Python script that creates a GUI with text boxes that accept the name of the item purchased, and the quantity. When the submit button is clicked, it calculates the total cost based on the price of the item. The code to connect with the price database to get the price of the item that is purchased is also written. The GUI also has a button to create an invoice copy based on the item, quantity purchased, and price. Then there is a close button, which asks where the invoice copy is required by creating a popup. After getting the user input, either closes or prints the invoice and then closes.\n\n```', additional_kwargs={}, example=False)

In [ ]:
import ast 

example_reply = """[
"main.py",
"gui.py",
"database.py",
"invoice.py"
]"""

ast.literal_eval(example_reply)

['main.py', 'gui.py', 'database.py', 'invoice.py']

In [ ]:
import ast

default_dir = 'generated'
file = None

list_actual = []

try:
    list_actual = ast.literal_eval(filepaths_string)

    # if shared_dependencies.md is there, read it in, else set it to None
    shared_dependencies = None
    if os.path.exists("shared_dependencies.md"):
        with open("shared_dependencies.md", "r") as shared_dependencies_file:
            shared_dependencies = shared_dependencies_file.read()

    if file is not None:
        # check file
        print("file", file)
        filename, filecode = generate_file(file, 
                                           model=chatLLM, 
                                           filepaths_string=filepaths_string, shared_dependencies=shared_dependencies, prompt=prompt)
        write_file(filename, filecode, default_dir)
    else:
        clean_dir(default_dir)

        # understand shared dependencies
        shared_dependencies = generate_response.call(model, 
            """You are an AI developer who is trying to write a program that will generate code for the user based on their intent.
            
        In response to the user's prompt:

        ---
        the app is: {prompt}
        ---
        
        the files we have decided to generate are: {filepaths_string}

        Now that we have a list of files, we need to understand what dependencies they share.
        Please name and briefly describe what is shared between the files we are generating, including exported variables, data schemas, id names of every DOM elements that javascript functions will use, message names, and function names.
        Exclusively focus on the names of the shared dependencies, and do not add any other explanation.
        """,
            prompt,
        )
        print(shared_dependencies)